<a href="https://colab.research.google.com/github/thanaprondtn/bsc_dpdm24/blob/main/Model_Thal_Group_653020570_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score,classification_report, confusion_matrix

In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/Data_model/train_set.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Data_model/X_test.csv')

In [4]:
train_df

,id number,Xn,Hb(Gm%),Hct(%),MCV(fl),A2 (2-3%),HbF (2%),HbA (95-97%),RBC,MCH,MCHC,MCV-RBC-3*HGB,MCV-RBC-5*HGB,|80-MCV|,|27-MCH|,|80-MCV|*|27-MCH|,Thal_Group
0,67d169,25.0,11.1,34.0,78.0,3.2,0.0,55.9,4.358974,25.464706,32.647059,40.341026,18.141026,2.0,1.535294,3.070588,Abnormal_variants
1,67l168,32.0,13.9,42.0,86.0,3.7,0.3,54.8,5.197674,28.859060,33.557047,35.802326,5.802326,6.0,1.859060,11.154362,Abnormal_variants
2,67i267,43.0,10.1,31.0,78.0,2.6,0.3,56.2,3.974359,25.412903,32.580645,43.725641,23.525641,2.0,1.587097,3.174194,Abnormal_variants
3,67e248,33.0,12.9,38.0,76.0,3.0,0.2,61.2,5.000000,25.800000,33.947368,32.300000,6.500000,4.0,1.200000,4.800000,Abnormal_variants
4,67h277,23.0,12.9,40.0,78.0,3.1,0.0,55.6,5.128205,25.155000,32.250000,34.171795,8.371795,2.0,1.845000,3.690000,Abnormal_variants
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,66z431,63.0,10.1,31.0,83.0,2.8,1.8,84.2,3.734940,27.041935,32.580645,48.965060,28.765060,3.0,0.041935,0.125806,Normal
4318,66z436,19.0,11.1,34.0,79.0,2.6,0.6,78.0,4.303797,25.791176,32.647059,41.396203,19.196203,1.0,1.208824,1.208824,Normal
4319,66z439,80.0,10.9,33.9,79.0,3.0,0.2,87.2,4.291139,25.401180,32.153392,42.008861,20.208861,1.0,1.598820,1.598820,Normal
4320,66z445,27.0,13.2,41.4,87.0,2.9,0.3,87.6,4.758621,27.739130,31.884058,42.641379,16.241379,7.0,0.739130,5.173913,Normal


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4322 entries, 0 to 4321
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id number          4322 non-null   object 
 1   Xn                 3887 non-null   float64
 2   Hb(Gm%)            4315 non-null   float64
 3   Hct(%)             4315 non-null   float64
 4   MCV(fl)            4315 non-null   float64
 5   A2 (2-3%)          4189 non-null   float64
 6   HbF (2%)           4206 non-null   float64
 7   HbA (95-97%)       4206 non-null   float64
 8   RBC                4315 non-null   float64
 9   MCH                4315 non-null   float64
 10  MCHC               4315 non-null   float64
 11  MCV-RBC-3*HGB      4315 non-null   float64
 12  MCV-RBC-5*HGB      4315 non-null   float64
 13  |80-MCV|           4315 non-null   float64
 14  |27-MCH|           4315 non-null   float64
 15  |80-MCV|*|27-MCH|  4315 non-null   float64
 16  Thal_Group         4322 

In [6]:
train_df.isnull().sum()

,0
id number,0
Xn,435
Hb(Gm%),7
Hct(%),7
MCV(fl),7
A2 (2-3%),133
HbF (2%),116
HbA (95-97%),116
RBC,7
MCH,7


In [7]:
test_df

,id number,Xn,Hb(Gm%),Hct(%),MCV(fl),A2 (2-3%),HbF (2%),HbA (95-97%),RBC,MCH,MCHC,MCV-RBC-3*HGB,MCV-RBC-5*HGB,|80-MCV|,|27-MCH|,|80-MCV|*|27-MCH|
0,66z133,38.0,12.4,37.00,78.0,2.7,1.3,75.4,4.743590,26.140541,33.513514,36.056410,11.256410,2.0,0.859459,1.718919
1,66z297,28.0,12.8,39.40,82.0,2.7,0.0,97.3,4.804878,26.639594,32.487310,38.795122,13.195122,2.0,0.360406,0.720812
2,66u491,78.0,10.9,33.90,79.0,3.1,0.0,96.9,4.291139,25.401180,32.153392,42.008861,20.208861,1.0,1.598820,1.598820
3,66x072,65.0,10.9,33.90,79.0,3.0,0.2,87.9,4.291139,25.401180,32.153392,42.008861,20.208861,1.0,1.598820,1.598820
4,67m031,17.0,11.5,36.00,78.0,2.8,0.8,79.6,4.615385,24.916667,31.944444,38.884615,15.884615,2.0,2.083333,4.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992,67j317,NaN,8.4,26.00,59.0,2.6,0.7,0.3,4.406780,19.061538,32.307692,29.393220,12.593220,21.0,7.938462,166.707692
1993,66a385,NaN,10.1,31.00,78.0,3.3,0.0,55.0,3.974359,25.412903,32.580645,43.725641,23.525641,2.0,1.587097,3.174194
1994,67g355,53.0,10.4,34.00,77.0,3.0,0.2,49.6,4.415584,23.552941,30.588235,41.384416,20.584416,3.0,3.447059,10.341176
1995,66s121,54.0,10.1,31.00,78.0,2.4,0.3,42.0,3.974359,25.412903,32.580645,43.725641,23.525641,2.0,1.587097,3.174194


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1997 entries, 0 to 1996
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id number          1997 non-null   object 
 1   Xn                 1810 non-null   float64
 2   Hb(Gm%)            1995 non-null   float64
 3   Hct(%)             1995 non-null   float64
 4   MCV(fl)            1996 non-null   float64
 5   A2 (2-3%)          1935 non-null   float64
 6   HbF (2%)           1938 non-null   float64
 7   HbA (95-97%)       1938 non-null   float64
 8   RBC                1995 non-null   float64
 9   MCH                1995 non-null   float64
 10  MCHC               1995 non-null   float64
 11  MCV-RBC-3*HGB      1995 non-null   float64
 12  MCV-RBC-5*HGB      1995 non-null   float64
 13  |80-MCV|           1996 non-null   float64
 14  |27-MCH|           1995 non-null   float64
 15  |80-MCV|*|27-MCH|  1995 non-null   float64
dtypes: float64(15), object(1

In [9]:
test_df.isnull().sum()

,0
id number,0
Xn,187
Hb(Gm%),2
Hct(%),2
MCV(fl),1
A2 (2-3%),62
HbF (2%),59
HbA (95-97%),59
RBC,2
MCH,2


In [10]:
# ลบคอลัมน์ที่ไม่จำเป็น
train_df.drop(columns=['id number'], inplace=True, errors='ignore')
test_df.drop(columns=['id number'], inplace=True, errors='ignore')

In [11]:
# แยก features และ target
X = train_df.drop(columns=['Thal_Group'])
y = train_df['Thal_Group']

In [12]:
# ตรวจหาค่า Missing Values และเติมค่าด้วยค่ามัธยฐานของแต่ละคอลัมน์
X.fillna(X.median(), inplace=True)
test_df.fillna(X.median(), inplace=True)

In [13]:
# Label Encoding for categorical columns
object_cols = X.select_dtypes(include=['object']).columns
label_encoders = {}
for col in object_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    test_df[col] = le.transform(test_df[col].astype(str))
    label_encoders[col] = le

In [14]:
# Train-Test Split with Stratified Sampling
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [15]:
# Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(test_df)

In [16]:
# Define models
models = {
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Naive Bayes': GaussianNB()
}


In [17]:
# Evaluate Models using validation set
model_scores = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_val_scaled)
    f1 = f1_score(y_val, y_pred, average='weighted')  # Weighted F1-score
    model_scores[name] = f1

print("Validation Set Scores:")
print(model_scores)


Validation Set Scores:
{'Decision Tree': 0.9044037599238207, 'Random Forest': 0.9323604955069806, 'Naive Bayes': 0.8352276313524472}


In [18]:
# เพิ่มการประเมิน Random Forest อย่างละเอียด
rf_model = models['Random Forest']
y_val_pred = rf_model.predict(X_val_scaled)

In [19]:
# คำนวณ F1 score บน validation set
val_f1 = f1_score(y_val, y_val_pred, average='weighted')
print(f"\nRandom Forest Validation F1 Score: {val_f1:.4f}")


Random Forest Validation F1 Score: 0.9324


In [20]:
# แสดงรายงานการจำแนกประเภทโดยละเอียดสำหรับ validation set
print("\nClassification Report (Validation Set):")
print(classification_report(y_val, y_val_pred))


Classification Report (Validation Set):
                    precision    recall  f1-score   support

 Abnormal_variants       1.00      0.75      0.86         4
Alpha_thal_related       0.83      0.90      0.86       192
 Beta_thal_related       0.98      0.99      0.98        88
       HbE_related       0.99      0.99      0.99       205
            Normal       0.95      0.91      0.93       376

          accuracy                           0.93       865
         macro avg       0.95      0.91      0.92       865
      weighted avg       0.93      0.93      0.93       865



In [21]:
# แสดงเมทริกซ์ความสับสนสำหรับ validation set
print("\nConfusion Matrix (Validation Set):")
print(confusion_matrix(y_val, y_val_pred))


Confusion Matrix (Validation Set):
[[  3   0   0   0   1]
 [  0 172   1   3  16]
 [  0   1  87   0   0]
 [  0   1   1 202   1]
 [  0  34   0   0 342]]


In [22]:
# Perform Cross Validation on the entire dataset
print("\nCross-Validation Scores:")
# Define stratified cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Scale the entire dataset for cross-validation
X_scaled = scaler.fit_transform(X)

# ประเมิน Random Forest ผ่าน cross-validation อย่างละเอียด
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
cv_f1_scores = cross_val_score(rf_model, X_scaled, y, cv=cv, scoring='f1_weighted')

print("\nRandom Forest Cross-Validation F1 Scores by Fold:")
for i, score in enumerate(cv_f1_scores):
    print(f"Fold {i+1}: F1 = {score:.4f}")

print(f"Mean CV F1 Score: {cv_f1_scores.mean():.4f}")
print(f"Standard Deviation: {cv_f1_scores.std():.4f}")



Cross-Validation Scores:


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(



Random Forest Cross-Validation F1 Scores by Fold:
Fold 1: F1 = 0.9278
Fold 2: F1 = 0.9351
Fold 3: F1 = 0.9347
Fold 4: F1 = 0.9278
Fold 5: F1 = 0.9244
Mean CV F1 Score: 0.9300
Standard Deviation: 0.0042


In [23]:
# Train the Random Forest model (best model) on the full training data
best_model = RandomForestClassifier(n_estimators=100, random_state=42)
best_model.fit(X_scaled, y)


RandomForestClassifier(random_state=42)

In [24]:
# Make predictions on test set
test_predictions = best_model.predict(X_test_scaled)

In [25]:
# จัดรูปแบบผลลัพธ์ให้ตรงกับ y_pred_baseline.csv
submission = pd.DataFrame({'Thal_Group': test_predictions})
submission.to_csv('/content/drive/MyDrive/Data_model/y_pred_baseline_653020570-8.csv', index=False)
print('Saved predictions to y_test_pred.csv')

Saved predictions to y_test_pred.csv


In [26]:
# Calculate class distribution in predictions
prediction_counts = pd.Series(test_predictions).value_counts(normalize=True) * 100
print("\nClass Distribution in Test Predictions:")
for cls, pct in prediction_counts.items():
    print(f"Class {cls}: {pct:.2f}%")


Class Distribution in Test Predictions:
Class Normal: 42.61%
Class HbE_related: 23.99%
Class Alpha_thal_related: 23.03%
Class Beta_thal_related: 10.12%
Class Abnormal_variants: 0.25%


In [27]:
print(submission)

              Thal_Group
0                 Normal
1                 Normal
2                 Normal
3                 Normal
4                 Normal
...                  ...
1992  Alpha_thal_related
1993   Abnormal_variants
1994   Abnormal_variants
1995   Abnormal_variants
1996         HbE_related

[1997 rows x 1 columns]
